In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer


/scratch/user/siweicui/.conda/envs/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from neural_compressor.config import PostTrainingQuantConfig
from neural_compressor.quantization import fit

In [3]:
# https://huggingface.co/mistralai/Mixtral-8x7B-Instruct-v0.1
float_model = AutoModelForCausalLM.from_pretrained("mistralai/Mixtral-8x7B-Instruct-v0.1", cache_dir="/scratch/user/siweicui/LLM/huggingface")


Loading checkpoint shards: 100%|██████████| 19/19 [03:07<00:00,  9.88s/it]


In [6]:
model_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_id)
messages = [
    {"role": "user", "content": "What is your favourite condiment?"},
    {"role": "assistant", "content": "Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!"},
    {"role": "user", "content": "Do you have mayonnaise recipes?"}
]

inputs = tokenizer.apply_chat_template(messages, return_tensors="pt")

outputs = float_model.generate(inputs, max_new_tokens=5)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] What is your favourite condiment? [/INST]Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen! [INST] Do you have mayonnaise recipes? [/INST] Yes, I do have


In [7]:
float_model

MixtralForCausalLM(
  (model): MixtralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MixtralDecoderLayer(
        (self_attn): MixtralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MixtralRotaryEmbedding()
        )
        (block_sparse_moe): MixtralSparseMoeBlock(
          (gate): Linear(in_features=4096, out_features=8, bias=False)
          (experts): ModuleList(
            (0-7): 8 x MixtralBLockSparseTop2MLP(
              (w1): Linear(in_features=4096, out_features=14336, bias=False)
              (w2): Linear(in_features=14336, out_features=4096, bias=False)
              (w3): Linear(in_features=4096, out_features=14336, bias=False)
        

In [8]:
# from transformers import AutoModelForCausalLM, AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained("mistralai/Mixtral-8x7B-Instruct-v0.1")


tokenizer_config.json: 100%|██████████| 1.46k/1.46k [00:00<00:00, 4.73MB/s]
tokenizer.model: 100%|██████████| 493k/493k [00:00<00:00, 16.6MB/s]
tokenizer.json: 100%|██████████| 1.80M/1.80M [00:00<00:00, 11.2MB/s]
special_tokens_map.json: 100%|██████████| 72.0/72.0 [00:00<00:00, 267kB/s]


In [8]:
woq_conf = PostTrainingQuantConfig(approach="weight_only")


In [9]:
quantized_model = fit(model=float_model, conf=woq_conf)


2024-03-18 14:56:34 [INFO] Start auto tuning.
2024-03-18 14:56:34 [INFO] Quantize model without tuning!
2024-03-18 14:56:34 [INFO] Quantize the model with default configuration without evaluating the model.                To perform the tuning process, please either provide an eval_func or provide an                    eval_dataloader an eval_metric.
2024-03-18 14:56:34 [INFO] Adaptor has 5 recipes.
2024-03-18 14:56:34 [INFO] 0 recipes specified by user.
2024-03-18 14:56:34 [INFO] 3 recipes require future tuning.
2024-03-18 14:56:34 [INFO] *** Initialize auto tuning
2024-03-18 14:56:34 [INFO] {
2024-03-18 14:56:34 [INFO]     'PostTrainingQuantConfig': {
2024-03-18 14:56:34 [INFO]         'AccuracyCriterion': {
2024-03-18 14:56:34 [INFO]             'criterion': 'relative',
2024-03-18 14:56:34 [INFO]             'higher_is_better': True,
2024-03-18 14:56:34 [INFO]             'tolerable_loss': 0.01,
2024-03-18 14:56:34 [INFO]             'absolute': None,
2024-03-18 14:56:34 [INFO]     

In [12]:
messages = [
    {"role": "user", "content": "What is your favourite condiment?"},
    {"role": "assistant", "content": "Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!"},
    {"role": "user", "content": "Do you have mayonnaise recipes?"}
]

inputs = tokenizer.apply_chat_template(messages, return_tensors="pt")#.to("cuda")


In [ ]:
quantized_model

PyTorchFXModel(
  (_model): MixtralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MixtralDecoderLayer(
        (self_attn): MixtralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MixtralRotaryEmbedding()
        )
        (block_sparse_moe): MixtralSparseMoeBlock(
          (gate): Linear(in_features=4096, out_features=8, bias=False)
          (experts): ModuleList(
            (0-7): 8 x MixtralBLockSparseTop2MLP(
              (w1): Linear(in_features=4096, out_features=14336, bias=False)
              (w2): Linear(in_features=14336, out_features=4096, bias=False)
              (w3): Linear(in_features=4096, out_features=14336, bias=False)
           

In [ ]:
# https://github.com/intel/neural-compressor/issues/1208
quantized_model.model

MixtralModel(
  (embed_tokens): Embedding(32000, 4096)
  (layers): ModuleList(
    (0-31): 32 x MixtralDecoderLayer(
      (self_attn): MixtralAttention(
        (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
        (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
        (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
        (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        (rotary_emb): MixtralRotaryEmbedding()
      )
      (block_sparse_moe): MixtralSparseMoeBlock(
        (gate): Linear(in_features=4096, out_features=8, bias=False)
        (experts): ModuleList(
          (0-7): 8 x MixtralBLockSparseTop2MLP(
            (w1): Linear(in_features=4096, out_features=14336, bias=False)
            (w2): Linear(in_features=14336, out_features=4096, bias=False)
            (w3): Linear(in_features=4096, out_features=14336, bias=False)
            (act_fn): SiLU()
          )
        )
      )
      (input_l

In [ ]:
quantized_model.fp32_model.eval()
# 

MixtralModel(
  (embed_tokens): Embedding(32000, 4096)
  (layers): ModuleList(
    (0-31): 32 x MixtralDecoderLayer(
      (self_attn): MixtralAttention(
        (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
        (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
        (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
        (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        (rotary_emb): MixtralRotaryEmbedding()
      )
      (block_sparse_moe): MixtralSparseMoeBlock(
        (gate): Linear(in_features=4096, out_features=8, bias=False)
        (experts): ModuleList(
          (0-7): 8 x MixtralBLockSparseTop2MLP(
            (w1): Linear(in_features=4096, out_features=14336, bias=False)
            (w2): Linear(in_features=14336, out_features=4096, bias=False)
            (w3): Linear(in_features=4096, out_features=14336, bias=False)
            (act_fn): SiLU()
          )
        )
      )
      (input_l

In [ ]:
# quantized_model.model.generate()

In [ ]:
# quantized_model

# outputs = quantized_model.model(inputs)


In [ ]:
quantized_model.save("/scratch/user/siweicui/mix_precision/quantized_model")

In [ ]:
# https://github.com/huggingface/transformers/blob/main/src/transformers/modeling_outputs.py
outputs = quantized_model.model.generate(inputs, max_new_tokens=5)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))



MoeModelOutputWithPast(last_hidden_state=tensor([[[-0.5204, -0.1014, -1.5529,  ..., -0.5523, -0.0827,  2.7109],
         [ 4.7656, -6.9212,  0.1959,  ..., -0.4684,  0.7749, -8.4018],
         [ 7.7237,  0.7646,  3.1338,  ...,  0.5119,  4.0246,  1.1077],
         ...,
         [ 3.5215, -0.3258,  0.1427,  ...,  1.6548,  7.2612,  1.4007],
         [ 0.2919, -3.4817,  0.3647,  ...,  5.8540,  9.2899, -2.6923],
         [ 1.4423, -2.0224, -2.7371,  ..., -4.2999, 11.3563, -1.4674]]],
       grad_fn=<MulBackward0>), past_key_values=((tensor([[[[ 1.0314e-02, -8.6396e-02, -4.2948e-02,  ..., -7.2125e-01,
           -2.4895e+00, -3.2909e+00],
          [ 3.5587e+00, -3.0896e+00, -1.8836e+00,  ...,  1.6693e+00,
            9.7373e-01, -1.2376e+00],
          [ 4.8267e+00, -5.6102e+00, -3.1196e-01,  ...,  1.5126e+00,
            1.2588e+00, -5.0005e-01],
          ...,
          [-4.2980e+00,  3.3370e+00, -3.1120e+00,  ...,  1.7578e+00,
            9.9101e-01, -1.4429e+00],
          [-4.6417e+00, 

In [29]:
# type(outputs)

transformers.modeling_outputs.MoeModelOutputWithPast

In [45]:
# quantized_model
# q_model = AutoModelForCausalLM.from_pretrained(quantized_model)
# outputs2 = quantized_model.model(inputs)

PyTorchFXModel(
  (_model): MixtralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MixtralDecoderLayer(
        (self_attn): MixtralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MixtralRotaryEmbedding()
        )
        (block_sparse_moe): MixtralSparseMoeBlock(
          (gate): Linear(in_features=4096, out_features=8, bias=False)
          (experts): ModuleList(
            (0-7): 8 x MixtralBLockSparseTop2MLP(
              (w1): Linear(in_features=4096, out_features=14336, bias=False)
              (w2): Linear(in_features=14336, out_features=4096, bias=False)
              (w3): Linear(in_features=4096, out_features=14336, bias=False)
           

In [ ]:
# print(tokenizer.decode(outputs.last_hidden_state[0], skip_special_tokens=True))